In [1]:
from bs4 import BeautifulSoup
from urllib.parse import urlparse
import requests
from IPython import embed

In [2]:
base_url = "https://www.finn.no"
index_url = f"{base_url}/realestate/homes/search.html?location=0.20061&sort=PUBLISHED_DESC"
#page = requests.get(index_url)
#soup = BeautifulSoup(page.content, 'html.parser')

def get_soup(url):
    page = requests.get(url)
    return BeautifulSoup(page.content, 'html.parser')

soup = get_soup(index_url)
with open("index.html", "w") as f:
    f.write(soup.__str__())

In [4]:
print(soup.prettify())

<!DOCTYPE html>
<html lang="no">
 <head>
  <link href="https://www.finn.no/favicon.ico" rel="icon"/>
  <link href="https://www.finn.no/favicon-t-32x32.png" rel="icon" sizes="32x32"/>
  <link href="https://www.finn.no/favicon-t-192x192.png" rel="icon" sizes="192x192"/>
  <link href="https://www.finn.no/favicon-t-96x96.png" rel="icon" sizes="96x96"/>
  <link href="https://www.finn.no/favicon-t-16x16.png" rel="icon" sizes="16x16"/>
  <link href="https://www.finn.no/apple-touch-icon-180x180.png" rel="apple-touch-icon"/>
  <meta content="#06befb" name="msapplication-TileColor"/>
  <meta content="https://www.finn.no/mstile-144x144.png" name="msapplication-TileImage"/>
  <meta content="#06befb" name="theme-color"/>
  <meta content="https://www.finn.no/browserconfig.xml" name="msapplication-config"/>
  <link href="https://www.finn.no/manifest.json" rel="manifest"/>
  <script>
   (function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({'gtm.start':
        new Date().getTime(),event:'gtm.js'});var f=d.get

In [6]:
card = cards[0]
card.h2.a["href"]

'/realestate/homes/ad.html?finnkode=201988281'

In [7]:
def extract_int(s) -> int:
    return int("".join(digit for digit in s if digit.isdigit()))

# url_from_Cards
card.find_all("a")[0]["href"]
relative_urls = {card.h2.get_text(): card.h2.a["href"] for card in cards}

In [10]:
# find all pages

def find_all_pages(soup, start_side: int, base_url):
    pages = []
    current_page = start_side
    for page in soup.find_all("a", {"pagination__page button button--pill"}):
        curent_page = extract_int(page["aria-label"])
        if curent_page > start_side:
            pages.append(page)
    if len(pages) == 0:
        return []
    new_soup = get_soup(f"{base_url}{pages[-1]['href']}" )
    # embed(header="just created new soup")
    return pages + find_all_pages(
        new_soup, curent_page, base_url)
    
pages = find_all_pages(soup, start_side=0, base_url="https://www.finn.no/realestate/homes/search.html")
pages

[<a aria-current="page" aria-label="Side 1" class="pagination__page button button--pill" href="?location=0.20061&amp;sort=PUBLISHED_DESC">1</a>,
 <a aria-label="Side 2" class="pagination__page button button--pill" href="?location=0.20061&amp;sort=PUBLISHED_DESC&amp;page=2">2</a>,
 <a aria-label="Side 3" class="pagination__page button button--pill" href="?location=0.20061&amp;sort=PUBLISHED_DESC&amp;page=3">3</a>,
 <a aria-label="Side 4" class="pagination__page button button--pill" href="?location=0.20061&amp;sort=PUBLISHED_DESC&amp;page=4">4</a>,
 <a aria-label="Side 5" class="pagination__page button button--pill" href="?location=0.20061&amp;sort=PUBLISHED_DESC&amp;page=5">5</a>,
 <a aria-label="Side 6" class="pagination__page button button--pill" href="?location=0.20061&amp;sort=PUBLISHED_DESC&amp;page=6">6</a>,
 <a aria-label="Side 7" class="pagination__page button button--pill" href="?location=0.20061&amp;sort=PUBLISHED_DESC&amp;page=7">7</a>,
 <a aria-label="Side 8" class="paginati

In [57]:
# extract data from listing
kw = "definition-list definition-list--cols1to2"
url = "https://www.finn.no/realestate/homes/ad.html?finnkode=203500388"
soup = get_soup(url)
dl_table = soup.find("dl", {"class": kw})
data = {dt.get_text(): dt.find_next("dd").get_text() for dt in dl_table.find_all("dt")}
dts = dl_table.find_all("dt")
dds = [dt.find_next("dd") for dt in dts]

In [58]:
data

{'Boligtype': 'Leilighet',
 'Eieform bolig': 'Andel',
 'Soverom': '2',
 'Primærrom': '67 m²',
 'Bruksareal': '67 m²',
 'Etasje': '4',
 'Byggeår': '1956',
 'Energimerking': '\n\n                            G - rød\n\n'}

In [40]:
itr = dl_table.__iter__()


In [41]:
dt = next(itr)

In [48]:
print(dt)b

KeyError: 2

## New session


In [1]:
from typing import List
from bs4 import BeautifulSoup
from urllib.parse import urlparse
import requests
import pandas as pd
import util
from IPython import embed
from finn import *


In [2]:
base_url = "https://www.finn.no"
search_url = f"{base_url}/realestate/homes/search.html"
index_url = f"{search_url}?location=0.20061&sort=PUBLISHED_DESC"

In [3]:
# get index
base_url = "https://www.finn.no"
search_url = f"{base_url}/realestate/homes/search.html"
index_url = f"{search_url}?location=0.20061&sort=PUBLISHED_DESC"
index_soup = get_soup(index_url)

In [4]:
# get all pages
page_links = get_all_page_links(
    index_soup, start_side=0, base_url="https://www.finn.no/realestate/homes/search.html")

In [5]:
# get all cards
card_links = set()
for page_link in page_links:
    for card_link in get_card_links(get_soup(search_url + page_link)):
        if "http" in card_link:
            card_links.add(card_link)
        # making all links absolute,because not all was.
        else:
            card_links.add("https://www.finn.no" + card_link)
        # embed()

In [6]:
print(len(card_links))
print(len(set(card_links)))

1110
1110


In [7]:
# get data from cards
cards_data = []
N = len(card_links)
N_failed = 0
for i, card_link in enumerate(list(card_links)[:10]):
    try:
        cards_data.append(
            get_data(card_link)
        )
    except AttributeError as e:
        print(f"url: {card_link}\n{e}")
        print(e)
        N_failed += 1
    # p = (i/N)*100
    print(f"{(i/N)*100}. {i} out of {N}, {N_failed=}")


0.0. 0 out of 1110, N_failed=0
0.09009009009009009. 1 out of 1110, N_failed=0
0.18018018018018017. 2 out of 1110, N_failed=0
0.2702702702702703. 3 out of 1110, N_failed=0
0.36036036036036034. 4 out of 1110, N_failed=0
url: https://www.finn.no/eiendom/nybygg/prosjekt?finnkode=200636262&location=0.20061&sort=PUBLISHED_DESC&page=4
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
0.45045045045045046. 5 out of 1110, N_failed=1
0.5405405405405406. 6 out of 1110, N_failed=1
0.6306306306306306. 7 out of 1110, N_failed=1
0.7207207207207207. 8 out of 1110, N_failed=1
url: https://www.finn.no/eiendom/nybygg/prosjekt?finnkode=188693993&location=0.20061&sort=PUBLISHED_DESC&page=19
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
0.8108108108108109. 9 out of 1110, N_failed=2


In [8]:
df = pd.DataFrame.from_dict(cards_data)

In [16]:
df["price"]



0     4 580 000 kr
1     4 490 000 kr
2     8 800 000 kr
3    10 700 000 kr
4     3 650 000 kr
5     5 700 000 kr
6     4 150 000 kr
7    13 500 000 kr
Name: price, dtype: object